
Import the libraries






In [2]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

I define an array, URL and the HTML parser, look for the table and iterated for each row and value to append to the postal_codes array

After that I cast the array to a Data Frame Pandas, Rename the columns, filtered for 'Not assigned' in Borough column Join the values 
through semi colon Reset index data frame Update the Neighbourhood column for the Borough

In [3]:
postal_codes_array = []
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, "html.parser")

postal_table = soup.find('table', {"class" : "wikitable sortable"})

for row in postal_table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 3:
        postal_codes_array.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))

df = pd.DataFrame(postal_codes_array)
df.columns = ['PostalCode', 'Borough','Neighbourhood']
df = df[df.Borough != 'Not assigned']
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join)
df = df.reset_index()
df.loc[df['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = df['Borough']

In [4]:
df.shape

(103, 3)

In [ ]:
!wget -q -O 'canada_data.csv' https://cocl.us/Geospatial_data

df_coordinates = pd.read_csv("canada_data.csv") 
df_coordinates.columns = ['PostalCode', 'Latitude','Longitude']
df_coordinates = pd.merge(df, df_coordinates, on='PostalCode', how='inner')
df_coordinates